In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import sys
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
cwd = os.getcwd()
path = os.path.join(cwd, 'images')
outpath = os.path.join(cwd, 'images_gray')

i=0
catagories = []
#256*192
#512*384
df = pd.DataFrame(columns = np.arange(1152*864)+1)

def process_image(file,outpath):
    img = cv2.imread(file,0)
    img = np.array((img)).reshape(1152,864).astype(np.float32)
    
#     # Resizing image to maintain aspect ratio
#     basewidth = 512
#     wpercent = (basewidth/float(img.shape[0]))
#     hsize = int((float(img.shape[1])*float(wpercent)))
#     #print(hsize)
#     img = cv2.resize(img, (hsize,basewidth), interpolation = Image.ANTIALIAS)
    img = img.flatten()
    
    return img 

for filename in os.listdir(path):    
    img = process_image(os.path.join(path, filename),outpath)
    catagories.append(int(filename[7:9]))
    df.loc[i] = img
    i = i+1

df['category'] = np.array(catagories)

In [3]:
df.shape

(55, 995329)

In [4]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,995320,995321,995322,995323,995324,995325,995326,995327,995328,category
0,245.0,245.0,244.0,245.0,245.0,245.0,245.0,244.0,244.0,244.0,...,14.0,13.0,14.0,15.0,15.0,14.0,14.0,15.0,15.0,11
1,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,31.0,27.0,31.0,37.0,44.0,47.0,48.0,50.0,53.0,8
2,232.0,232.0,232.0,232.0,233.0,233.0,233.0,234.0,233.0,233.0,...,126.0,126.0,126.0,127.0,128.0,129.0,130.0,130.0,130.0,4
3,234.0,234.0,234.0,235.0,235.0,235.0,235.0,234.0,236.0,235.0,...,11.0,12.0,11.0,12.0,14.0,15.0,13.0,12.0,12.0,5
4,186.0,185.0,185.0,187.0,187.0,186.0,187.0,189.0,188.0,187.0,...,6.0,6.0,8.0,7.0,7.0,9.0,8.0,7.0,10.0,6


In [5]:
X = np.array(df.drop(['category'],axis=1))
y = np.array(df['category'])
print(X.shape)
print(y.shape)

(55, 995328)
(55,)


In [6]:
class PCA(object):
    
    def __init__(self,X,y):
        self.X = X
        self.y = y
        self.create_split(0.2)
        self.generate_eig_val_vec()
    
    def create_split(self,test_size):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size)
        
    def generate_eig_val_vec(self):
        adj = (self.X_train - self.X_train.mean()).astype(float)
        cov = np.matmul(adj,adj.T)
        self.eig_value, self.eig_vec = np.linalg.eig(cov)
    
    def generate_k_eig_val_vec(self,k):
        indices_sorted = np.argsort(self.eig_value)
        k_eig_vec = self.eig_vec[:,indices_sorted[::-1][:k]]
        k_eig_values = self.eig_value[indices_sorted[::-1]][:k]
        
        return k_eig_values, k_eig_vec    
    
    def generate_projection(self,k):
        adj = (self.X_train - self.X_train.mean()).astype(float)
        k_eig_values, k_eig_vec = self.generate_k_eig_val_vec(k)
        phi = np.matmul(k_eig_vec.T, adj)
        w = np.matmul(phi, adj.T)
        
        adj_test = (self.X_test - self.X_test.mean()).astype(float)
        proj_test_face = np.matmul(phi,adj_test.T)
        
        return w, proj_test_face
    
    def classify(self,w,wi):
        pred_list = []
        for column in wi.T:
            min_dist = np.linalg.norm(column - w.T[0])
            cat = self.y_train[0]
            index = 0
            for i,col in enumerate(w.T):
                dist = np.linalg.norm(column-col)
                if dist < min_dist:
                    min_dist = dist
                    index = i
                    cat = df.loc[i,'category']
            pred_list.append((min_dist,cat))
        
        self.calculate_accuracy(pred_list,self.y_test)
    
    def calculate_accuracy(self,y_pred,y):
        c=0
        for i,actual in enumerate(y):
            print(actual,y_pred[i][1])
            if actual == y_pred[i][1]:
                c = c+1
        print('Accuracy:',c/len(y_pred)*100,'%')

In [7]:
k = 40
pca = PCA(X,y)
w,wi = pca.generate_projection(k)
y_pred = pca.classify(w,wi)

11 9
3 7
4 7
11 11
6 2
1 1
2 3
8 1
2 2
5 10
1 4
Accuracy: 27.27272727272727 %
